In [56]:
# importing sqlite to connect to the database
# importing pandas for data manipulation
# importing requests to get data from the api
import pandas as pd
import sqlite3
import numpy as np
import requests

In [57]:
# Feature Engineering for FPL predictiions
# Creating predictive feature from player data

# Connecting to the database
conn = sqlite3.connect('fpl_data.db')

# Geting all the data
# querying the database to get player information form gameweek table
query = '''
    SELECT p.player_id, p.name, p.position, p.team,
           g.gameweek, g.minutes, g.goals_scored, g.assists,
           g.clean_sheets, g.bonus, g.total_points
    FROM gameweek_data g
    JOIN players p ON g.player_id = p.player_id
    ORDER BY p.player_id, g.gameweek
'''

# storing the data that was got from the query in a dataframe(table)
df = pd.read_sql_query(query, conn)
# closing the connection
conn.close()

# loading the whole dataframe/table
# Count number of players in the dataframe & the gameweek
print(f"Loaded {len(df)} records")
print(f"Players: {df['player_id'].nunique()}")
print(f"Gameweeks: {df['gameweek'].min()} to {df['gameweek'].max()}")
# printing out some of the dataframe table
print(df.head())

Loaded 17362 records
Players: 803
Gameweeks: 1 to 23
   player_id  name position     team  gameweek  minutes  goals_scored  \
0          1  Raya       GK  Arsenal         1       90             0   
1          1  Raya       GK  Arsenal         2       90             0   
2          1  Raya       GK  Arsenal         3       90             0   
3          1  Raya       GK  Arsenal         4       90             0   
4          1  Raya       GK  Arsenal         5       90             0   

   assists  clean_sheets  bonus  total_points  
0        0             1      3            10  
1        0             1      0             6  
2        0             0      0             2  
3        0             1      0             6  
4        0             0      0             2  


In [58]:
# Feature for rolling Averages 
# gets the players recent form from the lat 5 games
def rollingAVG(df, window=5):
    """ 
    Calculating the rolling average of total points for each player

    DataFrame with player data
    window number od previous games to average

    returns the average rolling average points
    """
    # window set in the parameter of the function
    print(f"Rolling average - last {window} games")

    # Sorting the data by player & gameweek
    df = df.sort_values(['player_id', 'gameweek'])
    
    # Calculating the rolling averages for each player, total points column, last 5 games
    # rolling average is calculated after the last 5 games 
    # .mean calculates the average
    # .shift(1) uses the previous games
    df['rolling_avg_points'] = df.groupby('player_id')['total_points'].transform(
        lambda x: x.rolling(window=window, min_periods=window).mean().shift(1)
    )

    # filling nan values with 0
    # e.g. gw 1-5 has no rolling average as not enough games have been played 
    # nan would break ml models so it is replaced with a 0 
    df['rolling_avg_points'] = df['rolling_avg_points'].fillna(0)
    
    # returning the daataframe with rolling averages 
    return df

# storing the data in a dataframe
df = rollingAVG(df, window=5)

# showing how the rolling averages works 
print("\nRolling average data")
print("\nHaaland's form:")
eg = df[df['name'].str.contains('Haaland', na=False)][
    ['name', 'gameweek', 'total_points', 'rolling_avg_points']
].head(10)
print(eg)

# checking the correlation between rolling average & points scored
# looking to see that the players with good form socre more points
print("\nCorrelation with points")
corr = df['rolling_avg_points'].corr(df['total_points'])
print(f"r = {corr:.3f}")

if corr > 0.5:
    print("Strong correlation, good form predicts more points")
elif corr > 0.3:
    print("Moderate correlation, form is a useful predictor")
else:
    print("Weak correlation")

Rolling average - last 5 games

Rolling average data

Haaland's form:
         name  gameweek  total_points  rolling_avg_points
9867  Haaland         1            13                 0.0
9868  Haaland         2             2                 0.0
9869  Haaland         3             9                 0.0
9870  Haaland         4            13                 0.0
9871  Haaland         5             9                 0.0
9872  Haaland         6            16                 9.2
9873  Haaland         7             8                 9.8
9874  Haaland         8            13                11.0
9875  Haaland         9             2                11.8
9876  Haaland        10            13                 9.6

Correlation with points
r = 0.416
Moderate correlation, form is a useful predictor


In [59]:
# feature that predicts player performance based on
# the oppenet the face on the upcomig gameweek
# players will predict higher points if they face a weak team

# getting the team name from the api 
# mapping the team id to the team name
print("Getting team mappings")

# getting data that holds all the teams info
bootstrap_url = "https://fantasy.premierleague.com/api/bootstrap-static/"
bootstrap_data = requests.get(bootstrap_url).json()

# creating a dictionary to map the team id's to the team name 
GetTeamName = {team['id']: team['name'] for team in bootstrap_data['teams']}

# printing all the teams
print(f"{len(GetTeamName)} teams")

Getting team mappings
20 teams


In [60]:
# getting the fixtures of what team plays who
print("Getting fixtures")

# getting the fixtures from the FPL API
fixtures_url = "https://fantasy.premierleague.com/api/fixtures/"
fixtures_data = requests.get(fixtures_url).json()

# creating a fixtures dataframe
# to store the data that was got from the query in a dataframe(table)
fixtures_list = []
for fixture in fixtures_data:
    # only has the games that are assigned event
    if fixture['event']:
        # home team's view
        fixtures_list.append({
            'gameweek': fixture['event'],
            'team': GetTeamName[fixture['team_h']],      
            'opponent': GetTeamName[fixture['team_a']],  
            'is_home': True
        })
        # away team's view
        fixtures_list.append({
            'gameweek': fixture['event'],
            'team': GetTeamName[fixture['team_a']], 
            'opponent': GetTeamName[fixture['team_h']],
            'is_home': False
        })

# converting the list to a dataframe table
fixtures_df = pd.DataFrame(fixtures_list)

# amount of fixtures it gets 
# printing out the fixtures for gameweek 1
print(f"{len(fixtures_df)} fixtures")
print("\nfixtures GW1")
print(fixtures_df[fixtures_df['gameweek'] == 1].head(10))

Getting fixtures
760 fixtures

fixtures GW1
   gameweek         team     opponent  is_home
0         1    Liverpool  Bournemouth     True
1         1  Bournemouth    Liverpool    False
2         1  Aston Villa    Newcastle     True
3         1    Newcastle  Aston Villa    False
4         1     Brighton       Fulham     True
5         1       Fulham     Brighton    False
6         1        Spurs      Burnley     True
7         1      Burnley        Spurs    False
8         1   Sunderland     West Ham     True
9         1     West Ham   Sunderland    False


In [61]:
# Seeing how strong a teams defeense is
# a team with a weak defense has a high difficulty rating & vice versa

# get the player data with rolling averages
conn = sqlite3.connect('fpl_data.db')
df = pd.read_sql_query("SELECT * FROM features", conn)
conn.close()

# calculate the average points scored against each team
# the teams that conceded more points are easier to play 
teamDefense = df.groupby('team')['total_points'].mean().reset_index()
teamDefense.columns = ['team', 'avg_points_conceded']

# creating a difficulty rating 1 - 10 for each team
# 1 means the team has a strong defense & dosent concede many points
# while 10 means the team has a weak defense* concedes a lot of points
minPts = teamDefense['avg_points_conceded'].min()
maxPts = teamDefense['avg_points_conceded'].max()

# making the scale from 1-10
teamDefense['difficulty_rating'] = (
    ((maxPts - teamDefense['avg_points_conceded']) / (maxPts - minPts)) * 9 + 1
).round(1)


# for every game, put a difficulty rating on oppopnent based on their defense
# creating a dictionary with the team name and difficulty
opponentDifficulty = dict(zip(teamDefense['team'], teamDefense['difficulty_rating']))

# adding the opponent difficulty to the fixtures
fixtures_df['opponent_difficulty'] = fixtures_df['opponent'].map(opponentDifficulty)

# printing out the fixtures dataframe table for arsenal of their first 5 games
# with the opponent difficulty 
print(fixtures_df[fixtures_df['team'] == 'Arsenal'].head())



    gameweek     team       opponent  is_home  opponent_difficulty
17         1  Arsenal        Man Utd    False                  6.9
30         2  Arsenal          Leeds     True                  5.8
57         3  Arsenal      Liverpool    False                  5.2
60         4  Arsenal  Nott'm Forest     True                  8.0
98         5  Arsenal       Man City     True                  1.0


In [62]:
# Check fixtures_df columns
print("Columns in fixtures_df:")
print(fixtures_df.columns.tolist())

print("\nSample fixtures_df:")
print(fixtures_df.head())

print("\nCheck if is_home exists:")
print('is_home' in fixtures_df.columns)

Columns in fixtures_df:
['gameweek', 'team', 'opponent', 'is_home', 'opponent_difficulty']

Sample fixtures_df:
   gameweek         team     opponent  is_home  opponent_difficulty
0         1    Liverpool  Bournemouth     True                  8.5
1         1  Bournemouth    Liverpool    False                  5.2
2         1  Aston Villa    Newcastle     True                  5.3
3         1    Newcastle  Aston Villa    False                  8.0
4         1     Brighton       Fulham     True                  3.3

Check if is_home exists:
True


In [63]:
# adding the oponent difficulty rating to each players gameweek data
# combineing the fixtures with the player data
#
print("\nChecking for opponent difficulty and home advantage")

# checking if columns already exist
if 'opponent_difficulty' in df.columns and 'is_home' in df.columns:
    print("opponent_difficulty and is_home already in dataframe")
    print("merge done already")
    
    # Just ensure is_home is int
    df['is_home'] = df['is_home'].astype(int)
    df['opponent_difficulty'] = df['opponent_difficulty'].astype(float)
    
else:
    print("Adding opponent difficulty and home advantage...")
    
    # combining fixtures data
    df = df.merge(
        fixtures_df[['gameweek', 'team', 'opponent', 'opponent_difficulty', 'is_home']], 
        on=['gameweek', 'team'],
        how='left'
    )
    
    print("Merged fixture data")

# converting is_home from true/false to 1/0 
# boolean to int
df['is_home'] = df['is_home'].astype(int)

# make any missing values default to 5.0 difficulty
# & making any missing is_home values 0
# handles edge cases where data is missing
df['opponent_difficulty'] = df['opponent_difficulty'].fillna(5.0)
df['is_home'] = df['is_home'].fillna(0).astype(int)

# statistics of home & away records
records = len(df)
home_records = df['is_home'].sum()
away_records = records - home_records

print(f"Records: {records}")
print(f"Home games: {home_records} ({home_records/records*100:.1f}%)")
print(f"Away games: {away_records} ({away_records/records*100:.1f}%)")

# checking home advantage
home_avg = df[df['is_home'] == 1]['total_points'].mean()
away_avg = df[df['is_home'] == 0]['total_points'].mean()
print(f"\nAverage points at home: {home_avg:.2f}")
print(f"Average points away: {away_avg:.2f}")
print(f"Home advantage: +{home_avg - away_avg:.2f} points")

# e.g. for haaland
print("\nHaaland's fixtures with opponent difficulty")
haaland = df[df['name'].str.contains('Haaland', na=False)][
    ['name', 'gameweek', 'team', 'opponent', 'total_points', 
     'rolling_avg_points', 'opponent_difficulty']
].head(10)
print(haaland)

# checking the correlation between oponent difficulty & points scored
# looking to see that the players that face easy opponents score more points
print("\nCorrelation with points:")

opp_corr = df['opponent_difficulty'].corr(df['total_points'])
print(f"opponent difficulty: r = {opp_corr:.3f}")

if opp_corr > 0.0:
    print("+ correlation shows easier opponents lead to more points!")
elif opp_corr < 0.0:
    print("- correlation shows an unexpected pattern")
else:
    print("Weak correlation means the feature is not implemented properly")

# checking is_home correlation to see if players get more points when they play at home
is_home_corr = df['is_home'].corr(df['total_points'])
print(f"is_home: r = {is_home_corr:.3f}")

if is_home_corr > 0.05:
    print("Players score more points at home")
elif is_home_corr > 0.0:
    print("Slight home advantage detected")
else:
    print("No home advantage found")
"""
# saving the dataset with all features
print("\nSaving features")

# saving to the database for fast querying in Ml training
# if it exists replace new with the old data
conn = sqlite3.connect('fpl_data.db')
df.to_sql('features', conn, if_exists='replace', index=False)
conn.close()

print("Saved features")
print(f"Total records: {len(df)}")
"""


Checking for opponent difficulty and home advantage
opponent_difficulty and is_home already in dataframe
merge done already
Records: 16559
Home games: 8282 (50.0%)
Away games: 8277 (50.0%)

Average points at home: 1.30
Average points away: 1.13
Home advantage: +0.17 points

Haaland's fixtures with opponent difficulty
         name  gameweek      team     opponent  total_points  \
9438  Haaland         1  Man City       Wolves            13   
9439  Haaland         2  Man City        Spurs             2   
9440  Haaland         3  Man City     Brighton             9   
9441  Haaland         4  Man City      Man Utd            13   
9442  Haaland         5  Man City      Arsenal             9   
9443  Haaland         6  Man City      Burnley            16   
9444  Haaland         7  Man City    Brentford             8   
9445  Haaland         8  Man City      Everton            13   
9446  Haaland         9  Man City  Aston Villa             2   
9447  Haaland        10  Man City  Bourn

'\n# saving the dataset with all features\nprint("\nSaving features")\n\n# saving to the database for fast querying in Ml training\n# if it exists replace new with the old data\nconn = sqlite3.connect(\'fpl_data.db\')\ndf.to_sql(\'features\', conn, if_exists=\'replace\', index=False)\nconn.close()\n\nprint("Saved features")\nprint(f"Total records: {len(df)}")\n'

In [64]:
# feature to show that the minutes a player plays impacts their score
# a player who dosent play will score 0 points
# printing the minutes stats
print("Minutes stats")
print(df['minutes'].describe())

# checking the correlation between minutes played & points scored
# looking to see that players who play more score more points
print("\nCorrelation with points:")
corr = df['minutes'].corr(df['total_points'])
print(f"r = {corr:.3f}")

if corr > 0.5:
    print("Strong correlation, the more minutes played the more points scored")
else:
    print("Weak correlation")

# showing that the minutes played works
print("\nHaaland's minutes played v points scored")
haaland_minutes = df[df['name'].str.contains('Haaland', na=False)][
    ['name', 'gameweek', 'minutes', 'total_points']
].head(10)
print(haaland_minutes)


Minutes stats
count    16559.000000
mean        26.173682
std         37.689484
min          0.000000
25%          0.000000
50%          0.000000
75%         68.000000
max         90.000000
Name: minutes, dtype: float64

Correlation with points:
r = 0.685
Strong correlation, the more minutes played the more points scored

Haaland's minutes played v points scored
         name  gameweek  minutes  total_points
9438  Haaland         1       72            13
9439  Haaland         2       90             2
9440  Haaland         3       90             9
9441  Haaland         4       86            13
9442  Haaland         5       75             9
9443  Haaland         6       90            16
9444  Haaland         7       90             8
9445  Haaland         8       90            13
9446  Haaland         9       90             2
9447  Haaland        10       81            13


In [65]:
# price feature to show that expensive players score more points
# than less expensive players
print("Price")

# fpl api provides currennt player prices in the bootstrap-static endpoint
# the prices are stored as 'new_cost' in tenths
# a players new_cost that is 100 means they cost £10.0m
try:
    # getting data that holds all the players info
    print("\nGetting player prices from FPL API")
    url = "https://fantasy.premierleague.com/api/bootstrap-static/"
    response = requests.get(url)
    data = response.json()

    # extracting player prices from the api response
    # creating a dictionary to map the player id's to the player price
    print("Processing price data")
    price_dict = {}
    for player in data['elements']:
        player_id = player['id']
        # didviding by 10 to get the price in millions
        price = player['now_cost'] / 10.0
        price_dict[player_id] = price

    # adding the price column to the dataframe by mapping player_id to price_dict
    df['price'] = df['player_id'].map(price_dict)

    # handling missing values
    # some players may not have a price in the api data
    # filling any missing prices with the average price
    missing = df['price'].isna().sum()
    if missing > 0:
        avg_price = df['price'].mean()
        df['price'] = df['price'].fillna(avg_price)
        print(f"Filled {missing} missing prices with average price £{avg_price:.1f}m")

    # checking o see if the data was added correctly
    print(f"Range: £{df['price'].min():.1f}m - £{df['price'].max():.1f}m")
    print(f"Average: £{df['price'].mean():.1f}m")
    print(f"Median: £{df['price'].median():.1f}m")

    # check price distribution
    # showing how many players are in each price range
    # £4-6m budget players, £6-8m mid-range, £8-12m expensive, £12m+ elite
    print(f"\nPrice distribution:")
    print(f"Budget (£4-6m): {len(df[df['price'] < 6.0])} records")
    print(f"Mid (£6-8m): {len(df[(df['price'] >= 6.0) & (df['price'] < 8.0)])} records")
    print(f"Premium (£8-10m): {len(df[(df['price'] >= 8.0) & (df['price'] < 10.0)])} records")
    print(f"Elite (£10m+): {len(df[df['price'] >= 10.0])} records")

    # checking the correlation between player price & points scored
    # looking to see that expensive players score more points
    corr = df['price'].corr(df['total_points'])
    print(f"\nCorrelation with points: r = {corr:.3f}")

    # correlation strength
    if corr > 0.3:
        print("Strong correlation - expensive players score more!")
    elif corr > 0.2:
        print("Moderate correlation")
    else:
        print("Weak correlation")

    # Calculating average points per price bracket
    # shows the linear relationship between price & points scored
    print("\nAverage points per price bracket:")
    brackets = [(0, 5), (5, 6), (6, 7), (7, 8), (8, 10), (10, 20)]
    for low, high in brackets:
        bracket_df = df[(df['price'] >= low) & (df['price'] < high)]
        avg_points = bracket_df['total_points'].mean()
        print(f"£{low}-{high}m: {avg_points:.2f} points over {len(bracket_df)} records")

    # showing the top 5 most expensive players in df
    print("\nTop 5 most expensive players:")
    # drop_duplicates to show each player once (not every gameweek)
    top5 = df[['name', 'price']].drop_duplicates('name').nlargest(5, 'price')
    for _, row in top5.iterrows():
        print(f"{row['name']}: £{row['price']:.1f}m")
    
    # showing example data for haaland
    print("\nHaaland's price v points scored")
    haaland_price = df[df['name'].str.contains('Haaland', na=False)][
        ['name', 'gameweek', 'price', 'total_points']
    ].head(5)
    print(haaland_price.to_string(index=False))

except Exception as e:
    # if fpl api cannot be accessed 
    # adding a price colum filled with 0 tin  prevent breaking code
    # allowing the rest of the feature engineering to work
    print(f"Error accessing FPL API: {e}")
    df['price'] = 0

Price

Getting player prices from FPL API
Processing price data
Range: £3.7m - £15.1m
Average: £4.9m
Median: £4.7m

Price distribution:
Budget (£4-6m): 14634 records
Mid (£6-8m): 1573 records
Premium (£8-10m): 242 records
Elite (£10m+): 110 records

Correlation with points: r = 0.304
Strong correlation - expensive players score more!

Average points per price bracket:
£0-5m: 0.71 points over 10579 records
£5-6m: 1.85 points over 4055 records
£6-7m: 2.18 points over 1048 records
£7-8m: 3.14 points over 525 records
£8-10m: 2.86 points over 242 records
£10-20m: 3.65 points over 110 records

Top 5 most expensive players:
Haaland: £15.1m
M.Salah: £14.0m
Palmer: £10.4m
Isak: £10.3m
Saka: £10.1m

Haaland's price v points scored
   name  gameweek  price  total_points
Haaland         1   15.1            13
Haaland         2   15.1             2
Haaland         3   15.1             9
Haaland         4   15.1            13
Haaland         5   15.1             9


In [66]:
# player position feature
# checking how player position impacts points scored
# GK - poinnts from clean sheets & saves
# DEF - points from clean sheets & occasional goals
# MID - points from assists, goals & clean sheets
# FWD - points from goals & assists
print("\nPlayer position analysis")

# check to see if position column exists in dataframe
if 'position' not in df.columns:
    print("Position column not found in dataframe")

# if position column exists
else:
    # creating a one-hot encoding of the position column
    # one-hot encoding converts position into binary columns for each position
    # every player gets a 1 in their position column & 0 in the others
    df['pos_GK'] = (df['position'] == 'GK').astype(int)
    df['pos_DEF'] = (df['position'] == 'DEF').astype(int)
    df['pos_MID'] = (df['position'] == 'MID').astype(int)
    df['pos_FWD'] = (df['position'] == 'FWD').astype(int)

    # showing the records for each position
    # to see the encoding worked correctly
    print("\nPosition records:")
    print(f"GK: {df['pos_GK'].sum()} records")
    print(f"DEF: {df['pos_DEF'].sum()} records")
    print(f"MID: {df['pos_MID'].sum()} records")
    print(f"FWD: {df['pos_FWD'].sum()} records")

    # verifying total records equal total matches
    total_encoded = df['pos_GK'].sum() + df['pos_DEF'].sum() + df['pos_MID'].sum() + df['pos_FWD'].sum()
    print(f"Total encoded: {total_encoded} / {len(df)} records")

    # checking for errors players with no position or multiple positions
    if total_encoded != len(df):
        print(f"Some records not encoded properly")
    else:
        print(f"All records encoded correctly")

    # calculating average points per position
    # shows how different positions score differently
    print("\nAverage points per position:")
    positions = ['GK', 'DEF', 'MID', 'FWD']
    for pos in positions:
        pos_df = df[df['position'] == pos]
        if len(pos_df) == 0:
            print(f"{pos}: No records found")
            continue
        else:
            avg_points = pos_df['total_points'].mean()
            print(f"{pos}: {avg_points:.2f} points over {len(pos_df)} records")

    # showing one player from each position to validate encoding
    print(f"\nBinary encoding validation")
    print(f"{'Name':<20} {'Pos':<5} {'GK':<5} {'DEF':<5} {'MID':<5} {'FWD':<5}")
    print("-" * 55)
    for pos in positions:
        # Get one example from gameweek 1
        example = df[(df['position'] == pos) & (df['gameweek'] == 1)]
        if len(example) > 0:
            example = example.iloc[0]
            print(f"{example['name']:<20} {example['position']:<5} "
                  f"{example['pos_GK']:<5} {example['pos_DEF']:<5} "
                  f"{example['pos_MID']:<5} {example['pos_FWD']:<5}")
        else:
            print(f"No {pos} found in gameweek 1")

    # showing the highes scoring player from each position in one gameweek
    # shows that different postions scores differently
    print(f"\nTop scorer by position (Gameweek 1):")
    gw1 = df[df['gameweek'] == 1]
    
    for pos in positions:
        pos_gw1 = gw1[gw1['position'] == pos]
        if len(pos_gw1) > 0:
            top_player = pos_gw1.nlargest(1, 'total_points').iloc[0]
            print(f"   {pos}: {top_player['name']:<20} "
                  f"({top_player['total_points']:.0f} points)")
        else:
            print(f"{pos}: No data")



Player position analysis

Position records:
GK: 1908 records
DEF: 5464 records
MID: 7381 records
FWD: 1806 records
Total encoded: 16559 / 16559 records
All records encoded correctly

Average points per position:
GK: 0.78 points over 1908 records
DEF: 1.30 points over 5464 records
MID: 1.25 points over 7381 records
FWD: 1.26 points over 1806 records

Binary encoding validation
Name                 Pos   GK    DEF   MID   FWD  
-------------------------------------------------------
Raya                 GK    1     0     0     0    
Gabriel              DEF   0     1     0     0    
Saka                 MID   0     0     1     0    
Havertz              FWD   0     0     0     1    

Top scorer by position (Gameweek 1):
   GK: Raya                 (10 points)
   DEF: Ballard              (17 points)
   MID: Semenyo              (15 points)
   FWD: Haaland              (13 points)


In [67]:
# Clean_Sheets_Rolling_average feature
# tracks defensive performance of teams (useful for defenders & goalkeepers)
# players on teams with good defense get more clean sheets & score more points
print("\nClean Sheets Rolling Average Feature")

# check to see if clean_sheets column exists in dataframe
try:
    # if clean_sheets not in df.columns
    if 'clean_sheets' not in df.columns:
        print("clean_sheets column not found in dataframe")

        # get clean_sheets data from gameweek_data table
        conn = sqlite3.connect('fpl_data.db')
        query = '''
            SELECT player_id, gameweek, clean_sheets
            FROM gameweek_data
        '''
        cs_df = pd.read_sql_query(query, conn)
        conn.close()

        print(f"Getting {len(cs_df)} records from database")

        # merging clean_sheets data into main dataframe
        df = df.merge(
            cs_df,
            on=['player_id', 'gameweek'],
            how='left'
        )

        # Filling missing values with 0
        missing_cs = df['clean_sheets'].isna().sum()
        if missing_cs > 0:
            print(f"{missing_cs} records missing clean_sheets, filling with 0")
            df['clean_sheets'].fillna(0, inplace=True)

        print("clean_sheets data merged successfully")
    else:
        print("clean_sheets column already in dataframe")

    # calculating rolling average of clean sheets for each player (last 5 games)
    # shows if a players team has been defensively solid 
    # using .shift(1) to use previous games only
    print("\nCalculating clean_sheets rolling average (last 5 games)")
    df = df.sort_values(['player_id', 'gameweek'])

    # Calculating rolling average
    df['clean_sheets_rolling_avg'] = df.groupby('player_id')['clean_sheets'].transform(
        lambda x: x.rolling(window=5, min_periods=5).mean().shift(1)
    )
    
    # filling nan values 
    # using 0 for missing values
    df['clean_sheets_rolling_avg'] = df['clean_sheets_rolling_avg'].fillna(0)

    # showing basic stats for clean_sheets_rolling_avg
    print("\nClean Sheets Rolling Average Stats:")
    print(f"Range: {df['clean_sheets_rolling_avg'].min():.2f} - {df['clean_sheets_rolling_avg'].max():.2f}")
    print(f"Average: {df['clean_sheets_rolling_avg'].mean():.2f} clean sheets per game")
    print(f"Median: {df['clean_sheets_rolling_avg'].median():.2f} clean sheets per game")

    # showing distribution of clean_sheets_rolling_avg
    print(f"\nDefensive form Distribution:")
    excellent = len(df[df['clean_sheets_rolling_avg'] > 0.6])
    good = len(df[(df['clean_sheets_rolling_avg'] > 0.4) & (df['clean_sheets_rolling_avg'] <= 0.6)])
    average = len(df[(df['clean_sheets_rolling_avg'] > 0.2) & (df['clean_sheets_rolling_avg'] <= 0.4)])
    poor = len(df[df['clean_sheets_rolling_avg'] <= 0.2])

    print(f"Excellent (>60% clean sheets): {excellent} records")
    print(f"Good (40-60% clean sheets): {good} records")
    print(f"Average (20-40% clean sheets): {average} records")
    print(f"Poor (<20% clean sheets): {poor} records")

    # showing correlation for defenders & goalkeepers
    # more clean sheets = more points
    print("\nCorrelation with points:")

    if 'pos_DEF' in df.columns and 'pos_GK' in df.columns:
        # filtering for defenders & goalkeepers
        def_gk_df = df[(df['pos_DEF'] == 1) | (df['pos_GK'] == 1)]

        cs_corr = def_gk_df['clean_sheets_rolling_avg'].corr(def_gk_df['total_points'])
        print(f"Defenders & Goalkeepers: r = {cs_corr:.3f}")

        if cs_corr > 0.15:
            print("Positive correlation - solid defense leads to more points")
        elif cs_corr > 0.05:
            print("Moderate positive correlation")
        else:
            print("Weak correlation")

    # filtering for forwards
    if 'pos_FWD' in df.columns:
        fwd_df = df[df['pos_FWD'] == 1]

        if len(fwd_df) > 0:
            fwd_corr = fwd_df['clean_sheets_rolling_avg'].corr(fwd_df['total_points'])
            print(f"Forwards: r = {fwd_corr:.3f}")
            if abs(fwd_corr) < 0.05:
                print("Near zero correlation - defensive form not relevant for forwards")
            
    # comparing defenders from a top defensive team v a poor defensive team
    print(f"\nDefensive form impact (Gameweek 10):")
    gw10 = df[df['gameweek'] == 10]

    if len(gw10) > 0:
        # best defense with highest clean sheet rate
        best_def = gw10[gw10['position'] == 'DEF'].nlargest(5, 'clean_sheets_rolling_avg')[
            ['name', 'clean_sheets_rolling_avg', 'total_points']
        ]

        print(f"\nTop 5 defenders with the best defensive form:")
        for _, row in best_def.iterrows():
            cs_pct = row['clean_sheets_rolling_avg'] * 100
            print(f"{row['name']:<20} Clean sheet rate: {cs_pct:.0f}% → {row['total_points']:.0f} pts")
        
        # Worst defense with lowest clean sheet rate
        worst_def = gw10[gw10['position'] == 'DEF'].nsmallest(5, 'clean_sheets_rolling_avg')[
            ['name', 'clean_sheets_rolling_avg', 'total_points']
        ]
        
        print(f"\nBottom 5 defenders with the worst defensive form:")
        for _, row in worst_def.iterrows():
            cs_pct = row['clean_sheets_rolling_avg'] * 100
            print(f"{row['name']:<20} Clean sheet rate: {cs_pct:.0f}% → {row['total_points']:.0f} pts")

# adding a dummy column with values = 0 to prevent breaking code
except Exception as e:
    print(f"\nERROR: {e}")
    print("Skipping clean_sheets_rolling_avg feature")
    df['clean_sheets_rolling_avg'] = 0


Clean Sheets Rolling Average Feature
clean_sheets column already in dataframe

Calculating clean_sheets rolling average (last 5 games)

Clean Sheets Rolling Average Stats:
Range: 0.00 - 0.80
Average: 0.06 clean sheets per game
Median: 0.00 clean sheets per game

Defensive form Distribution:
Excellent (>60% clean sheets): 61 records
Good (40-60% clean sheets): 327 records
Average (20-40% clean sheets): 905 records
Poor (<20% clean sheets): 15266 records

Correlation with points:
Defenders & Goalkeepers: r = 0.307
Positive correlation - solid defense leads to more points
Forwards: r = 0.304

Defensive form impact (Gameweek 10):

Top 5 defenders with the best defensive form:
Gabriel              Clean sheet rate: 60% → 12 pts
Calafiori            Clean sheet rate: 60% → 6 pts
J.Timber             Clean sheet rate: 60% → 6 pts
O'Reilly             Clean sheet rate: 60% → 8 pts
Saliba               Clean sheet rate: 40% → 6 pts

Bottom 5 defenders with the worst defensive form:
Kiwior     

In [68]:
# saving the engineering features
print("\nFeature engineering complete, saving features")

# ml features
ml_features = [
    'rolling_avg_points',
    'opponent_difficulty',
    'minutes',
    'is_home',
    'price',
    'pos_GK',
    'pos_DEF',
    'pos_MID',
    'pos_FWD',
    'clean_sheets_rolling_avg'
]

# checking which features exist
print("\nML Features for Training:")
available_features = []
missing_features = []

for feature in ml_features:
    if feature in df.columns:
        print(f"{feature}")
        available_features.append(feature)
    else:
        print(f"{feature} - MISSING")
        missing_features.append(feature)

print(f"\nTotal ML features: {len(available_features)}/{len(ml_features)}")

if missing_features:
    print(f"{len(missing_features)} features missing")
    for feature in missing_features:
        print(f"{feature}")

print("FEATURE CORRELATIONS WITH TOTAL POINTS")
print("-"*80)

# checking features correlation with total points
correlation_features = [
    'rolling_avg_points',
    'opponent_difficulty',
    'minutes',
    'is_home',
    'price',
    'clean_sheets_rolling_avg'
]

correlations = []
print("\nPearson correlation coefficients:")
for feature in correlation_features:
    if feature in df.columns:
        corr = df[feature].corr(df['total_points'])
        correlations.append((feature, abs(corr), corr))
        print(f"{feature}: r = {corr:+.3f}")

# sorting by strongest correlation to lowest
correlations.sort(key=lambda x: x[1], reverse=True)

print(f"\nRanked by strength:")
for feature, abs_corr, corr in correlations:
    print(f"{feature}: r = {abs_corr:.3f}")

# saving to a csv file 
# index does not allow it to add extra row numbers 
df.to_csv('fpl_features.csv', index=False)
print("Saved features to 'fpl_features.csv'")

# saving to the database for fast querying in Ml training
# if it exists replace new with the old data
conn = sqlite3.connect('fpl_data.db')
df.to_sql('features', conn, if_exists='replace', index=False)
conn.close()
print("Saved features to database features")

print(f"\nFeature engineering done")
print(f"Total records: {len(df)}")
print(f"Feature: rollingAverages - last 5 games")
print(f"Feature: opponentDifficulty - based on team defense")
print(f"Feature: is_home - home advantage")
print(f"Feature: price - current player price")
print(f"Feature: position - one-hot encoding of player position")
print(f"Feature: clean_sheets_rolling_avg - last 5 games")


Feature engineering complete, saving features

ML Features for Training:
rolling_avg_points
opponent_difficulty
minutes
is_home
price
pos_GK
pos_DEF
pos_MID
pos_FWD
clean_sheets_rolling_avg

Total ML features: 10/10
FEATURE CORRELATIONS WITH TOTAL POINTS
--------------------------------------------------------------------------------

Pearson correlation coefficients:
rolling_avg_points: r = +0.415
opponent_difficulty: r = +0.051
minutes: r = +0.685
is_home: r = +0.036
price: r = +0.304
clean_sheets_rolling_avg: r = +0.322

Ranked by strength:
minutes: r = 0.685
rolling_avg_points: r = 0.415
clean_sheets_rolling_avg: r = 0.322
price: r = 0.304
opponent_difficulty: r = 0.051
is_home: r = 0.036
Saved features to 'fpl_features.csv'
Saved features to database features

Feature engineering done
Total records: 16559
Feature: rollingAverages - last 5 games
Feature: opponentDifficulty - based on team defense
Feature: is_home - home advantage
Feature: price - current player price
Feature: posi